# Structure your model

## Agenda
* Overall structure of a model in TensorFlow
* word2vec
* Name scope
* Embedding visualization

この時点までで、TensorFlowでの2つのシンプルなモデルを実装しました。
1つはシカゴの火災と窃盗の件数の線形回帰。
もう一つはロジスティック回帰を用いたMNISTのデータ・セットでの光学的な認識。
ツールを用いることで、より複雑なモデルのビルドができるようになりました。
しかし、複雑なモデルはより良い計画を要求します。もしくは、デバッグするのに少々厄介かもしれません。
次の2つの講義では、モデルの効果的な構成に関するディスカッションをします。
そして、word2vecの例を通じてそれを行います。

あなたが言葉の構造を最もよく精通し、wod2vecのようなモデルの重要性を理解している、とします。
あまり詳しくない型は[CS 224Nのスライド](http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture2.pdf)を参照しましょう。
オリジナルの論文は以下になります。
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1301.3781.pdf)
* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)

端的に言うと、単語のベクトル表現を必要とします
それはいくつかのマジックのトリックを行うためにNeural Networkにinputする必要があります。多くのモデル学習の基礎から得られた単語のベクトルはCS224N学ぶことができました。現実世界での言語モデルと同様に。

3D空間に単語のベクトル表現を描画することもできました。

先の講義では、word2vecをビルドすることを試みました。skip-gramモデルを使用して。
ここからskip-gram モデルの説明とチュートリアルを見つけることができます。
[Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)

skip-gramのモデルでは、単語のベクトル表現を求めるために、特定のタスクを実行するための単層隠れ層を持ったシンプルなニューラルネットワーク訓練します。しかし、そこでは私達は訓練されたタスクのために、ニューラルネットワークを使用しません。
その代わり、隠れ層の重みを気にします。これらの重みは学習するためのword vectors もしくは embedding matrixです。

私達がモデルとして学習しようとしている特定のフェークのタスクは与えられた中心の単語の近所を予測しています。文中の特定の単語を与えた場合、近くの単語を確認し、ランダムに1つピックアップします。ネットワークは私達が選択した近所の単語の語彙のすべての単語に対する可能性を教えてくれます。

### Softmax, Negative Sampling, and Noise Contrastive Estimation

CS 224Nでは、2つの訓練のメソッドを学びました。hierarchical softmaxとnegative samplingです。私達はsoftmaxを排除しました。それは正規化係数の計算コストが大きすぎるからです。そして、skip-gramのnegative samplingを学びました。

Negative samplingは、その名の通り、サンプリングに基づくアプローチの有名なものです。Negative samplingはNoise Contrastive Estimation(NCE)と呼ばれるアプローチの簡単なモデルです。

negative samplingはword embeddingsの学習に役立ちます。
Word Embeddingとは、単語間の意味上の関係性を捉えることです。

NCE は論理保証が良い。

Samplingベースのアプローチは、negative samplingかNCEに関わらず、訓練に時間がかかります。

### データセットに関して
text8 は 100MBのクリーンなテキストです。これは英語のWikipediaからdumpしたものです。クリーンなテキストを使用することで、生のテキストを処理する時間をTensorFlowに集中させることができます。

100MBは本当に良い word embeddingには十分ではありませんが、興味深い関係性を見るには十分です。17,005,207のトークンがスペースで区切られています。

良い結果を得るために、wikipediaのdumpの最初の10^9byteのfil9データセットを使用すべきです。これは[Matt Mahoney’s website](https://cs.fit.edu/~mmahoney/compression/textdata.html) で言及されています。

## どのようにしてTensorFlowのモデルを構築するか
たった2つのモデルしか触っていないし、それらは似たような構造をしている。

#### Phase1: グラフを組み立てる
1. inputとoutputのplaceholderを定義する
2. weightを定義する
3. 推論モデルを定義する
4. loss functionを定義する
5. optimizerを定義する

#### Phase2: 計算を実行する
モデルの基本的な訓練のために、いくつかのステップを踏みます。
1. 最初にすべてのvariablesを初期化します
2. 訓練用データを与えます。データサンプルの順序はランダムかもしれません。
3. 訓練データの推論モデルを実行します。
4. コストを計算します
5. モデルパラメータを調整します。


In [2]:
import tensorflow as tf

# 1: Define placeholders for input and output
# inputはcenterwords、outputはtargetwrords、one-hot vectorを使用する代わりに
# これらの単語のインデックスをインプットにします。
#例えば、center wordが語彙の1001番目なら、1001を入力します

center_words = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
target_words = tf.placeholder(tf.int32, shape=[BATCH_SIZE])

# 2: Define the weight
# 単語のベクトル表現に各行が対応しています。
# 1単語がEMBED_SIZEのベクトルで表現される場合、embedding matrix は [VOCAB_SIZE, EMBED_SIZE]のベクトrになります。
embed_matrix =  tf.Variables(tf.random_uniform([VOCAB_SIZE, EMBED_SIZE], -1.0, 1.0))

#3: Inference
tf.nn.embedding_lookup(embed_matrix, center_words)

#4: Define loss function
nce_weight = tf.variable(tf.truncated_normal([VOCAB_SIZE, EMBED_SIZE], stddev=1.0 / EMBED_SIZE ** 0.5))
nce_bias = tf.variable(tf.zeros([VOCAB_SIZE]))

loss = tf.reduce_mean(tf.nn.nce_loss(weight=nce_weight,
                                    biases=nce_bias,
                                    labels=target_words,
                                    inputs=embed,
                                    num_sampled=NUM_SAMPLED,
                                    num_classes=VOCAB_SIZE))

# 5: Define optimizer
optimizer = tf.train.GradientDesentOptimizer(LEARNING_RATE).minimize(loss)

## Phase2: Execute Optimization
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    average_loss = 0.0
    for index in xrange(NUM_RAIN_STEPS):
        batch = batch_gen.next()
        loss_batch, _ = sess.run([loss, optimizer],
                                feed_dict = {center_words: batch[0], target_words: batch[1]})
        average_loss += loss_batch
        if (index + 1) % 2000 == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, average_loss / (index + 1)))
        


NameError: name 'BATCH_SIZE' is not defined

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

from process_data import process_data

VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128 # dimension of the word embedding vectors
SKIP_WINDOW = 1 # the context window
NUM_SAMPLED = 64    # Number of negative examples to sample.
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 10000
SKIP_STEP = 2000 # how many steps to skip before reporting the loss

def word2vec(batch_gen):
    """ Build the graph for word2vec model and train it """
    # Step 1: define the placeholders for input and output
    # center_words have to be int to work on embedding lookup

    # TO DO

    # Step 2: define weights. In word2vec, it's actually the weights that we care about
    # vocab size x embed size
    # initialized to random uniform -1 to 1

    # TOO DO

    # Step 3: define the inference
    # get the embed of input words using tf.nn.embedding_lookup
    # embed = tf.nn.embedding_lookup(embed_matrix, center_words, name='embed')

    # TO DO

    # Step 4: construct variables for NCE loss
    # tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, ...)
    # nce_weight (vocab size x embed size), intialized to truncated_normal stddev=1.0 / (EMBED_SIZE ** 0.5)
    # bias: vocab size, initialized to 0

    # TO DO

    # define loss function to be NCE loss function
    # tf.nn.nce_loss(weights, biases, labels, inputs, num_sampled, num_classes, ...)
    # need to get the mean accross the batch

    # TO DO

    # Step 5: define optimizer

    # TO DO

    with tf.Session() as sess:
        # TO DO: initialize variables

        total_loss = 0.0 # we use this to calculate the average loss in the last SKIP_STEP steps
        writer = tf.summary.FileWriter('./my_graph/no_frills/', sess.graph)
        for index in xrange(NUM_TRAIN_STEPS):
            centers, targets = batch_gen.next()
            # TO DO: create feed_dict, run optimizer, fetch loss_batch

            total_loss += loss_batch
            if (index + 1) % SKIP_STEP == 0:
                print('Average loss at step {}: {:5.1f}'.format(index, total_loss / SKIP_STEP))
                total_loss = 0.0
        writer.close()

def main():
    batch_gen = process_data(VOCAB_SIZE, BATCH_SIZE, SKIP_WINDOW)
    word2vec(batch_gen)

if __name__ == '__main__':
    main()%